In [2]:
import sys
#sys.path.append("fashion-clip/")
from fashion_clip.fashion_clip import FashionCLIP
import numpy as np
from sklearn.metrics import *
import pandas as pd

In [3]:
# %%capture
fclip = FashionCLIP('fashion-clip')

In [4]:
annotations = pd.read_csv('D:\M1/fashion/lda_model/data/anotation_new.csv')
img_path = annotations.iloc[:, 0]
images = img_path.values

In [5]:
item_set = set()
new_images = []
for i in images:
    item_id = i.split('/')[-1]
    if item_id in item_set:
        continue
    new_images.append(i)
    item_set.add(item_id)
images = new_images

In [6]:
# images = images[:len(images)// 100]
images = [i.replace('\\', '/') for i in images]

In [8]:
image_embeddings = fclip.encode_images(images, batch_size=128)

  1%|▏         | 113/8079 [00:46<56:07,  2.37it/s]

KeyboardInterrupt: 

In [ ]:
image_embeddings = image_embeddings/np.linalg.norm(image_embeddings, ord=2, axis=-1, keepdims=True)

In [ ]:
category = ['Vest top', 'Hair/alice band', 'Leggings/Tights', 'T-shirt', 'Sneakers',
 'Sunglasses', 'Cardigan', 'Gloves', 'Underwear Tights', 'Hoodie', 'Other shoe',
 'Shorts' ,'Jumpsuit/Playsuit', 'Dress', 'Trousers', 'Belt', 'Socks',
 'Underwear bottom' ,'Bodysuit' ,'Hat/beanie', 'Scarf' ,'Jacket',
 'Other accessories', 'Bra' ,'Swimwear bottom' ,'Blazer' ,'Top' ,'Polo shirt',
 'Sweater' ,'Necklace' ,'Pyjama set' ,'Blouse', 'Bag', 'Shirt' ,'Coat' ,'Boots',
 'Skirt', 'Garment Set', 'Bikini top', 'Sandals' ,'Dungarees', 'Earring',
 'Cap/peaked' ,'Ballerinas', 'Swimsuit' ,'Hat/brim']
materials = ['Cotton', 'Polyester', 'Rayon', 'Linen', 'Wool', 'Silk', 'Nylon', 'Polyurethane', 'Denim', 'Spandex']
colors = ['Black', 'White', 'Gray', 'Navy', 'Blue', 'Red', 'Green', 'Brown', 'Beige', 'Pink',
          'Yellow', 'Orange', 'Purple', 'Lavender', 'Teal', 'Turquoise', 'Magenta', 'Olive', 'Maroon', 'Charcoal']

In [ ]:
created_material_labels = []
created_color_labels = []
for c in category:
    for color in colors:
        created_color_labels.append(f'{c} {color}')
    for m in materials:
        created_material_labels.append(f'{c} {m}')
created_color_labels

['Vest top Black',
 'Vest top White',
 'Vest top Gray',
 'Vest top Navy',
 'Vest top Blue',
 'Vest top Red',
 'Vest top Green',
 'Vest top Brown',
 'Vest top Beige',
 'Vest top Pink',
 'Vest top Yellow',
 'Vest top Orange',
 'Vest top Purple',
 'Vest top Lavender',
 'Vest top Teal',
 'Vest top Turquoise',
 'Vest top Magenta',
 'Vest top Olive',
 'Vest top Maroon',
 'Vest top Charcoal',
 'Hair/alice band Black',
 'Hair/alice band White',
 'Hair/alice band Gray',
 'Hair/alice band Navy',
 'Hair/alice band Blue',
 'Hair/alice band Red',
 'Hair/alice band Green',
 'Hair/alice band Brown',
 'Hair/alice band Beige',
 'Hair/alice band Pink',
 'Hair/alice band Yellow',
 'Hair/alice band Orange',
 'Hair/alice band Purple',
 'Hair/alice band Lavender',
 'Hair/alice band Teal',
 'Hair/alice band Turquoise',
 'Hair/alice band Magenta',
 'Hair/alice band Olive',
 'Hair/alice band Maroon',
 'Hair/alice band Charcoal',
 'Leggings/Tights Black',
 'Leggings/Tights White',
 'Leggings/Tights Gray',
 'Leg

In [ ]:
label_material_embeddings = fclip.encode_text(created_material_labels, batch_size=32)
label_material_embeddings = label_material_embeddings/np.linalg.norm(label_material_embeddings, ord=2, axis=-1, keepdims=True)
label_color_embeddings = fclip.encode_text(created_color_labels, batch_size=32)
label_color_embeddings = label_color_embeddings/np.linalg.norm(label_color_embeddings, ord=2, axis=-1, keepdims=True)

15it [00:01, 11.35it/s]                            
29it [00:02, 13.58it/s]                            


In [ ]:
# predicteds
predicted_materials = label_material_embeddings.dot(image_embeddings.T)
predicted_materials = [created_material_labels[k] for k in np.argmax(predicted_materials, axis=0)]
predicted_colors = label_color_embeddings.dot(image_embeddings.T)
predicted_colors = [created_color_labels[k] for k in np.argmax(predicted_colors, axis=0)]

In [ ]:
with open('D:\M1/fashion/lda_model/data/fashion_clip.txt', 'w') as f:
    for img_path, m, c in zip(images, predicted_materials, predicted_colors):
        # item_id = i.split('/')[-1]
        item_id = img_path.split('/')[-1][:-6]
        
        f.write(f'{item_id} {m} {c} \n')